In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

In [2]:
# Define the noise function
def add_noise(img, noise_factor=0.5):
    noise = torch.randn(img.size()) * noise_factor
    noisy_img = img + noise
    noisy_img = torch.clamp(noisy_img, 0., 1.)
    return noisy_img

# Define a custom dataset class to apply noise
class NoisyFashionMNIST(datasets.FashionMNIST):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        img = add_noise(img)
        return img, target

In [3]:
# Hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size = 100
num_classes = 10

# Load the noisy dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])
print(device)
model_save_path = "resnet18_fmnist.pth"  # Path to save model weights

cuda


In [6]:
# Define the target model
train_dataset_noisy = NoisyFashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset_noisy = NoisyFashionMNIST(root='./data', train=False, transform=transform)

train_loader_noisy = DataLoader(dataset=train_dataset_noisy, batch_size=batch_size, shuffle=True)
test_loader_noisy = test_loader = DataLoader(dataset=test_dataset_noisy, batch_size=batch_size, shuffle=False)

Using downloaded and verified file: ./data/NoisyFashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/NoisyFashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/NoisyFashionMNIST/raw

Using downloaded and verified file: ./data/NoisyFashionMNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/NoisyFashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/NoisyFashionMNIST/raw

Using downloaded and verified file: ./data/NoisyFashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/NoisyFashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/NoisyFashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.36MB/s]

Extracting ./data/NoisyFashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/NoisyFashionMNIST/raw



In [ ]:
target_model_noisy = torchvision.models.resnet18(pretrained=False)
target_model_noisy.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
target_model_noisy.fc = nn.Linear(target_model_noisy.fc.in_features, num_classes)
target_model_noisy = target_model_noisy.to(device)

In [ ]:
batch = next(iter(test_loader))
images, _ = batch

In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(target_model_noisy.parameters(), lr=0.001)

In [ ]:
# Retrain the target model with the noisy dataset
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader_noisy:
        images, labels = images.to(device), labels.to(device)
        outputs = target_model_noisy(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Save the retrained model
torch.save(target_model_noisy.state_dict(), 'resnet18_fmnist_noisy.pth')